# 1. Import Library

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import os
from dotenv import load_dotenv
from pathlib import Path

import re

import faiss
from openai import OpenAI

from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from langsmith import Client

d:\Hafiz\Kuliah\Semester 5\Information Retrieval\Projek\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_colwidth', 500)

# 2. Preparing Data

In [3]:
full_df = pd.read_csv('../artikel/raw_health_articles.csv')
df = full_df[['content', 'source']]
df.head()

,content,source
0,"Ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. Jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. Banyak penyakit ginjal apa saja yang berkembang tanpa gejala pada tahap awal, sehingga penting menjaga kesehatan ginjal sejak dini.\nBerikut ini adalah 5 cara merawat ginjal yang sederhana namun efektif untuk mencegah gejala ginjal bermasalah:\n1. Minum Air Putih yang Cukup\nAsupan cairan yang cukup mem...",Biofarma
1,"Menjaga kesehatan mata sangatlah penting karena mata berperan besar dalam hampir semua aktivitas sehari-hari, mulai dari belajar, bekerja, hingga berinteraksi dengan lingkungan sekitar. Jika kesehatan mata terabaikan, berbagai masalah bisa muncul, mulai dari iritasi ringan, infeksi, hingga penyakit serius yang berisiko menyebabkan kebutaan. Untuk itu, menjaga mata dapat dilakukan melalui pemeriksaan rutin ke dokter dan juga dengan membiasakan pola hidup sehat yang mendukung fungsi penglihata...",Biofarma
2,"Penyebab Campak\nCampak adalah penyakit menular yang disebabkan oleh virus Morbili dan dapat menyebar dengan sangat cepat dari satu orang ke orang lain. Mekanisme utama penularannya adalah melalui percikan air liur (droplet) yang keluar dari mulut atau hidung penderita saat batuk, bersin, atau berbicara. Selain itu, seseorang juga dapat tertular jika menyentuh benda yang sudah terkontaminasi virus lalu menyentuh hidung, mulut, atau mata. Berikut adalah sepuluh faktor penyebab yang paling ber...",Biofarma
3,"Apa itu campak?\nCampak atau rubella adalah penyakit menular yang disebabkan oleh virus dari keluarga Paramyxoviridae dengan genus Morbillivirus. Penyakit ini menyerang saluran pernapasan terlebih dahulu, lalu menimbulkan gejala sistemik seperti demam, batuk, pilek, sakit tenggorokan, mata merah, hingga ruam menyeluruh di kulit. Campak termasuk penyakit yang sangat menular, bisa berpindah dari orang ke orang hanya melalui udara, percikan air liur, atau benda yang terkontaminasi virus.\nApa s...",Biofarma
4,"Apa itu Kesehatan Gigi dan Mulut?\nKesehatan gigi dan mulut adalah kondisi ketika gigi, gusi, lidah, dan seluruh rongga mulut terjaga kebersihan serta fungsinya dengan baik. Mulut merupakan gerbang utama bagi makanan dan minuman yang masuk ke dalam tubuh, sekaligus organ yang berperan penting dalam proses pencernaan. Saat makanan masuk, sebagian akan tertinggal dalam mulut. Jika tidak dibersihkan, sisa tersebut bisa menjadi tempat berkembangnya bakteri. Oleh karena itu, kesehatan mulut tidak...",Biofarma


# 3. Convert to Dictionary

In [4]:
df_dict = df.to_dict()

# 4. Text Cleaning

In [5]:
def clean_text(text):
    # Case-folding
    text = text.lower()

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE) # Remove URLs
    text = re.sub(r'\@w+|\#','', text) # Remove mentions and hashtags
    text = re.sub(r'kompas\.com\s*[–-]\s*', '', text, flags=re.MULTILINE) # Remove [NAMA KOTA], kompas.com -
    text = re.sub(r'baca juga:.*', '', text, flags=re.MULTILINE) # Remove baca juga
    text = re.sub(r'^.*\nartikel kompas\.id$', '', text, flags=re.MULTILINE) # Remove artikel kompas.id
    
    # Remove the specific UI noise at the end of the text
    text = text.replace("bagikan:\ndownload", "")
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    return text

In [6]:
docs = []
for index, content in df_dict['content'].items():
    cleaned_content = clean_text(content)
    # We create a Document object. 
    # We store the 'index' in metadata so we can trace the source later.
    doc = Document(
        page_content=cleaned_content, 
        metadata={"source_id": index}
    )
    docs.append(doc)

print(f"Loaded {len(docs)} documents.")

Loaded 420 documents.


# 5. Chunking

In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ".", " ", ""] # Priorities for splitting
)

In [8]:
splits = text_splitter.split_documents(docs)
print(f"Created {len(splits)} chunks from {len(docs)} original documents.")

Created 2660 chunks from 420 original documents.


# 6. Embedding

In [9]:
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)

In [10]:
vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=embedding_model
)

# 7. Retrieve Document

In [11]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # Retrieve the top 5 most relevant chunks
)

In [12]:
# Test retrieval
query = "Bagaimana cara menjaga kesehatan ginjal?"
retrieved_docs = retriever.invoke(query)

In [13]:
print(f"Query: {query}")
for i, doc in enumerate(retrieved_docs):
    print(f"\n[Result {i+1}] (Source ID: {doc.metadata['source_id']})")
    print(doc.page_content[:200] + "...")

Query: Bagaimana cara menjaga kesehatan ginjal?

[Result 1] (Source ID: 37)
ginjal adalah organ penting yang bekerja tanpa henti untuk menyaring limbah, menjaga keseimbangan elektrolit, dan mengatur tekanan darah. sayangnya, banyak orang baru menyadari pentingnya fungsi ginja...

[Result 2] (Source ID: 15)
ginjal merupakan salah satu organ vital dalam tubuh yang bekerja tanpa henti untuk menyaring darah, membuang limbah, dan menjaga keseimbangan cairan. letaknya berada di sisi kanan dan kiri tubuh, tepa...

[Result 3] (Source ID: 0)
ginjal adalah organ vital yang berfungsi menyaring darah, membuang limbah, dan menjaga keseimbangan cairan tubuh. jika tidak dijaga, fungsi ginjal bisa menurun dan menimbulkan masalah serius. banyak p...

[Result 4] (Source ID: 223)
. jagalah kesehatan ginjalmu dengan menjaga hidrasi yang cukup, mengendalikan tekanan darah dan kadar gula darah, menjalani gaya hidup sehat, dan melakukan pemeriksaan kesehatan rutin. jika kamu memil...

[Result 5] (Source ID: 

# 8. Complete Retrieval Augmented Generation (RAG)

In [14]:
# Check API
env_path = Path('..') / '.env'
load_dotenv(dotenv_path=env_path)

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

if GEMINI_API_KEY:
    print("API Key ditemukan!")
else:
    print("API Key masih None. Periksa path file .env")

API Key ditemukan!


In [15]:
client = OpenAI(
    api_key = GEMINI_API_KEY,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

def generate_augmented_response(query):
  search_results = retriever.invoke(query)
  context = "\n\n".join([doc.page_content for doc in search_results])

  prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
  response = client.chat.completions.create(
      model="gemini-2.5-flash", # Using model gemini 2.5 flash
      messages=[
        {
          "role": "system",
          "content": "You are a helpful assistant."
        },
        {
          "role": "user",
          "content": prompt
        }
      ],
  )

  return response.choices[0].message.content

## 8.1 Factual Queries

In [16]:
query = "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin."
response = generate_augmented_response(query)
print(response)

Berikut adalah zat gizi dalam wortel yang bermanfaat untuk kesehatan mata, beserta bagian teks yang mendukungnya:

1.  **Lutein**
    *   **Bagian yang mendukung:** ". lutein memainkan peran penting dalam menjaga kesehatan mata."

2.  **Beta-karoten**
    *   **Bagian yang mendukung:** "wortel kaya akan beta-karoten, yang diubah tubuh menjadi vitamin a. vitamin ini sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan mata seperti rabun senja dan degenerasi makula."
    *   **Bagian lain yang mendukung:** "beta-karoten, karotenoid utama yang terdapat dalam wortel, dapat diubah menjadi vitamin a di dalam tubuh. nutrisi ini penting untuk kesehatan mata..."

3.  **Vitamin A** (diubah dari Beta-karoten)
    *   **Bagian yang mendukung:** "wortel kaya akan beta-karoten, yang diubah tubuh menjadi vitamin a. vitamin ini sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan m

## 8.2 Conceptual Understanding?

In [17]:
query = "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?"
response = generate_augmented_response(query)
print(response)

Penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal karena:

1.  **Garam berlebih dapat meningkatkan tekanan darah.** Tekanan darah tinggi merupakan salah satu faktor utama penyebab penyakit ginjal kronis.
2.  **Tekanan darah tinggi merusak pembuluh darah di ginjal.** Jika dibiarkan, tekanan darah tinggi dapat menghambat aliran darah ke ginjal dan menurunkan fungsinya.
3.  **Mengurangi garam membantu menjaga aliran darah ke ginjal tetap lancar.** Dengan demikian, fungsi ginjal dapat terlindungi.
4.  Konsumsi garam berlebih juga dapat menyebabkan retensi cairan dan garam di dalam tubuh, yang membuat volume darah bertambah dan tekanan di pembuluh darah meningkat.


## 8.3 Question Requiring Synthesis

In [18]:
query = "Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?"
response = generate_augmented_response(query)
print(response)

Berdasarkan teks yang diberikan, berikut adalah analisis dampak merokok terhadap tubuh, diurutkan mulai dari kepala hingga organ dalam dan sistem lainnya:

**I. Kepala dan Leher**

*   **Otak:**
    *   **Kerusakan:** Rusak akibat penyempitan pembuluh darah otak dan penurunan suplai oksigen ke otak.
    *   **Penyakit/Dampak:** Gangguan pada seluruh sistem tubuh (karena nikotin mengganggu seluruh sistem).
*   **Indera (umum):**
    *   **Kerusakan:** Rusak (tidak dijelaskan spesifik bagian mata).
    *   **Penyakit/Dampak:** Penurunan fungsi indera akibat penyempitan pembuluh darah dan penurunan suplai oksigen.
*   **Mulut & Bibir:**
    *   **Kerusakan:** Pengikisan selaput lendir.
    *   **Penyakit/Dampak:** Kanker mulut, kanker bibir (disebabkan oleh ampas tar dan panas dari asap).
*   **Tenggorokan (Laring) / Kerongkongan (Esofagus):**
    *   **Kerusakan:** Perubahan sel-sel normal menjadi sel ganas. Eugenol pada rokok kretek memberikan efek baal/mati rasa, sehingga asap dihirup 

## 8.4 Question Out-Of-Domain

For checking hallucination from LLM generation

In [19]:
query = "Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!"
response = generate_augmented_response(query)
print(response)

Informasi mengenai pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD tidak tersedia dalam teks yang diberikan. Teks tersebut membahas tentang kolaborasi strategis KlikDokter dengan berbagai pihak dalam sektor kesehatan dan pembayaran digital.


## 8.5 Question for Comparation Analysis

In [20]:
query = "gejala diabetes"
response = generate_augmented_response(query)
print(response)

Berikut adalah gejala umum diabetes yang perlu disadari:

*   Buang air kecil berlebihan
*   Rasa haus yang berlebihan sehingga menyebabkan minum banyak cairan
*   Penurunan berat badan (bisa drastis pada anak)
*   Rasa lelah yang terus menerus (atau anak mulai loyo)
*   Penyembuhan luka lambat
*   Banyak makan (khususnya pada anak)
*   Ngompol lagi (pada anak yang sebelumnya sudah tidak ngompol)


# 9. Evaluation

## 9.1 Retrieval Evaluation

In [21]:
ground_truth = {
    "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin.": [78, 96],
    "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?": [0, 15, 53, 190, 223],
    "Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?": [4, 15, 46, 54, 276]
}

In [25]:
results = {}
k = 5

print(f"--- Retrieval Evaluation (Precision@{k}) ---")

for query, relevant_ids in ground_truth.items():
    # 1. Retrieve
    retrieved_docs = retriever.invoke(query)
    
    # 2. Take ID from metadata
    retrieved_ids = [doc.metadata.get('source_id') for doc in retrieved_docs]
    
    print(f"Query: {query}")
    print(f"   Debug - Retrieved IDs: {retrieved_ids}")
    print(f"   Debug - Target IDs: {relevant_ids}")

    # 3. Calculate Precision
    relevant_retrieved = [i for i in retrieved_ids if i in relevant_ids]
    
    precision = len(relevant_retrieved) / len(retrieved_docs) if retrieved_docs else 0
    results[query] = precision
    
    print(f"Final Precision@{k} Score: {precision:.2f}\n")

--- Retrieval Evaluation (Precision@5) ---
Query: Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata! dan sebutkan bagian mana yang mendukung setiap poin.
   Debug - Retrieved IDs: [96, 96, 96, 96, 311]
   Debug - Target IDs: [78, 96]
Final Precision@5 Score: 0.80

Query: Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?
   Debug - Retrieved IDs: [15, 0, 53, 316, 190]
   Debug - Target IDs: [0, 15, 53, 190, 223]
Final Precision@5 Score: 0.80

Query: Analisis dampak merokok terhadap tubuh: organ apa saja yang rusak dan penyakit apa yang mungkin timbul mulai dari kepala (mata) hingga organ dalam (ginjal/jantung)?
   Debug - Retrieved IDs: [54, 54, 276, 54, 276]
   Debug - Target IDs: [4, 15, 46, 54, 276]
Final Precision@5 Score: 1.00



## 9.2 Langsmith Evaluation

In [26]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Health_RAG_Evaluation"
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY

In [27]:
client_ls = Client(api_key=LANGCHAIN_API_KEY)

try:
    list(client_ls.list_datasets(limit=1))
    print("✅ Authentication SUCCESS!")
except Exception as e:
    print("❌ Authentication FAILED!")
    print(e)

✅ Authentication SUCCESS!


In [28]:
# New pipeline for evaluation (same as before but using dict as input)

def rag_pipeline(inputs):
    """
    RAG function that accepts a dict input (from LangSmith)
    and returns a dict output (Answer + Documents).
    """
    query = inputs["question"]
    
    # 1. Retrieve
    retrieved_docs = retriever.invoke(query)
    
    # 2. Contextualize
    context = "\n\n".join([doc.page_content for doc in retrieved_docs])
    
    # 3. Generate
    prompt = f"Context: {context}\n\nQuestion: {query}\n\nAnswer:"
    
    response = client.chat.completions.create(
      model="gemini-2.5-flash", 
      messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
      ],
    )
    
    answer_text = response.choices[0].message.content
    
    return {
        "answer": answer_text,
        "documents": retrieved_docs
    }

In [29]:
# Define the Judge LLM (Using Gemini 2.5 Flash as the grader)
llm_judge = ChatOpenAI(
    model="gemini-2.5-flash",
    openai_api_base="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=GEMINI_API_KEY,
    temperature=0
)

### 9.2.1 Correctness: Response vs Reference Answer

In [30]:
# --- 1. Correctness Evaluator ---
class CorrectnessGrade(BaseModel):
    explanation: str = Field(description="Explain your reasoning for the score")
    correct: bool = Field(description="True if the answer is correct, False otherwise.")

def correctness(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    prompt = f"""You are a teacher grading a quiz.
    QUESTION: {inputs['question']}
    GROUND TRUTH: {reference_outputs['answer']}
    STUDENT ANSWER: {outputs['answer']}
    
    Grade based on factual accuracy relative to the ground truth.
    """
    grader = llm_judge.with_structured_output(CorrectnessGrade)
    grade = grader.invoke(prompt)
    return grade.correct

### 9.2.2 Groundedness: Response vs Retrieved Docs

In [31]:
# --- 2. Groundedness Evaluator (Hallucination Check) ---
class GroundedGrade(BaseModel):
    explanation: str = Field(description="Explain your reasoning for the score")
    grounded: bool = Field(description="True if the answer is supported by facts, False otherwise.")

def groundedness(inputs: dict, outputs: dict) -> bool:
    # Combine retrieved docs into a single string
    doc_string = "\n\n".join(doc.page_content for doc in outputs["documents"])
    
    prompt = f"""You are a teacher grading a quiz.
    FACTS: {doc_string}
    STUDENT ANSWER: {outputs['answer']}
    
    Ensure the STUDENT ANSWER is grounded in the FACTS.
    """
    grader = llm_judge.with_structured_output(GroundedGrade)
    grade = grader.invoke(prompt)
    return grade.grounded

### 9.2.3 Run Evaluation

In [32]:
client_ls = Client(api_key=LANGCHAIN_API_KEY)

dataset_name = "Health_RAG_Dataset_V1.1"

# 1. Check if dataset exists, if not create it
if not client_ls.has_dataset(dataset_name=dataset_name):
    dataset = client_ls.create_dataset(dataset_name=dataset_name)
    
    examples = [
        {
            "inputs": {"question": "Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata!"},
            "outputs": {"answer": "Wortel mengandung Beta-karoten yang diubah menjadi Vitamin A, serta Lutein. "},
        },
        {
            "inputs": {"question": "Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?"},
            "outputs": {"answer": "Garam berlebih meningkatkan tekanan darah. Tekanan darah tinggi merusak pembuluh darah ginjal, menghambat aliran darah, dan menurunkan fungsi ginjal."},
        },
        {
            "inputs": {"question": "Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!"},
            "outputs": {"answer": "Maaf, informasi tersebut tidak tersedia dalam konteks dokumen yang diberikan."},
        }
    ]
    
    client_ls.create_examples(
        dataset_id=dataset.id,
        examples=examples
    )
    print("Dataset created!")
else:
    print("Dataset already exists.")

# 2. Run the Evaluation
results = client_ls.evaluate(
    rag_pipeline,
    data=dataset_name,
    evaluators=[correctness, groundedness],
    experiment_prefix="gemini-rag-test"
)

Dataset already exists.
View the evaluation results for experiment: 'gemini-rag-test-2682c3be' at:
https://smith.langchain.com/o/aa841df8-5022-4144-a456-958c1b95a656/datasets/c8639e6a-ad82-4561-bcf2-d7a9ee70afaf/compare?selectedSessions=abaae13e-24c9-4d54-833e-4ae03705e689




3it [00:32, 10.73s/it]


In [33]:
results.to_pandas()

,inputs.question,outputs.answer,outputs.documents,error,reference.answer,feedback.correctness,feedback.groundedness,execution_time,example_id,id
0,Sebutkan pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD!,"Informasi mengenai pasal yang mengatur tentang pengadaan barang dan jasa dalam UUD tidak tersedia dalam teks yang diberikan. Teks tersebut membahas kolaborasi Klikdokter dengan Danamon, Jalin, dan Kalbe/Kimia Farma, serta peran Klikdokter dalam inisiatif smart city dan edukasi kesehatan.","[page_content='berita kesehatan klikdokter dan danamon jalin kolaborasi strategis untuk tingkatkan kesehatan pelanggan klikdokter, 31 okt 2023 ditinjau oleh tim medis klikdokter bagikan jakarta, 31 oktober 2023 - pt bank danamon indonesia tbk (“danamon”) dan klikdokter sebagai penyedia layanan healthtech di bawah naungan kalbe group dengan bangga mengumumkan rencana kerja sama strategis jangka panjang untuk mempromosikan layanan kesehatan dan mempermudah transaksi pembayaran rencana kerja sa...",None,"Maaf, informasi tersebut tidak tersedia dalam konteks dokumen yang diberikan.",True,True,3.439943,54981987-21d5-4f5b-ae9f-775102938a9d,019adaf0-82d8-7110-a4a1-87010ecc5aff
1,Mengapa penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal?,"Penderita hipertensi harus membatasi konsumsi garam untuk menjaga kesehatan ginjal karena:\n\n1. **Garam berlebih meningkatkan tekanan darah.** Asupan garam yang terlalu banyak dapat menyebabkan tekanan darah tinggi.\n2. **Tekanan darah tinggi merusak pembuluh darah di ginjal.** Jika dibiarkan, tekanan darah tinggi dapat merusak pembuluh darah kecil di ginjal.\n3. **Merusak pembuluh darah menghambat aliran darah ke ginjal dan menurunkan fungsinya.** Kerusakan ini dapat menghambat aliran d...","[page_content='. selain itu, asupan air yang cukup juga membantu memperlancar sirkulasi darah ke ginjal dan mencegah pembentukan batu ginjal. 2. batasi konsumsi garam terlalu banyak garam bisa menyebabkan tekanan darah tinggi yang pada akhirnya merusak pembuluh darah di ginjal. jika dibiarkan, tekanan darah tinggi dapat menghambat aliran darah ke ginjal dan menurunkan fungsinya. 3. pertahankan berat badan ideal obesitas meningkatkan risiko terkena penyakit seperti diabetes dan hipertensi, ya...",None,"Garam berlebih meningkatkan tekanan darah. Tekanan darah tinggi merusak pembuluh darah ginjal, menghambat aliran darah, dan menurunkan fungsi ginjal.",True,True,5.120506,83714f98-4281-453c-9fba-794730748ea8,019adaf0-ab75-7696-b8bb-370ad937143c
2,Sebutkan zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata!,"Berdasarkan teks, zat gizi yang terkandung dalam wortel yang bermanfaat untuk kesehatan mata adalah:\n\n* **Beta-karoten**: Diubah tubuh menjadi vitamin A.\n* **Vitamin A**: Sangat penting untuk menjaga kesehatan mata, membantu mempertahankan penglihatan yang baik, dan mencegah gangguan mata seperti rabun senja dan degenerasi makula.","[page_content='. lutein memainkan peran penting dalam menjaga kesehatan mata. likopen : ini adalah antioksidan merah terang yang ditemukan dalam sejumlah buah dan sayuran merah, termasuk wortel merah dan ungu. telah disarankan bahwa likopen mungkin memiliki efek perlindungan terhadap kanker dan penyakit kardiovaskular. proses memasak wortel memfasilitasi pelepasan likopen dan meningkatkan penyerapannya. poliasetilena : penelitian terbaru telah mengidentifikasi senyawa bioaktif dalam wortel y...",None,"Wortel mengandung Beta-karoten yang diubah menjadi Vitamin A, serta Lutein.",False,True,5.523030,b8032bd4-b504-4851-a597-b1984d15c57d,019adaf0-d4fc-748d-824b-29017ec2fc8e
